## vLLM(Virvual large Language Model) HuggingFace 모델 서버란?

vLLM은 **HuggingFace 기반의 LLM(대형 언어 모델)** 을 **빠르게 서빙(서비스 제공)** 하고 **Inference** 할 수 있게 도와주는 오픈소스이다. 비동기, 병렬적인 요청 처리와 KV 캐시 재사용에 초점을 맞춘 라이브러리이다.

- PagedAttention: 필요한 부분만 부분적으로 KV cache를 공유하고 로딩해서 VRAM 절약
- 비동기 추론: 다수의 요청을 빠르게 처리 
- OpenAI API 호환 서버: 클라이언트는 `openai.ChatCompletion.create()`처럼 요청 가능 

 Linux OS만 지원하기 때문에 vLLM 실습은 Ubuntu에서 실행

### vLLM 서버가 하는 일
- 모델 로딩
    - HuggingFace에서 원하는 모델을 다운로드하거나, 이미 캐시된 모델을 불러옵니다.

- 모델 최적화
    - 예: 모델을 float16처럼 더 가벼운 형식으로 변환해서 더 빠르게 실행되도록 만듭니다.

- 파이프라인 구성
    - 모델을 실행 가능한 형태로 로드합니다. (추론이 가능한 형태로 세팅)

- 성능 최적화
    - 파이프라인 중 일부를 더 빠른 연산 방식(예: fused ops)으로 바꿔 성능을 개선합니다.

- API 서버로 제공
    - 모델에 접근할 수 있는 표준화된 HTTP API 엔드포인트를 열어둡니다.


### vLLL environment

가상환경 실행
```bash
source ./vllm-env/bin/activate```

PyTorch 설치 (CUDA 11.8 기준)
```bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
```

vLLM 설치
```bash
pip install vllm
```

Optional: transformers와 accelerate도 설치
```bash
pip install transformers accelerate
```

---
vllm 서버 실행
```bash
python3 -m vllm.entrypoints.openai.api_server \
    --model TinyLlama/TinyLlama-1.1B-Chat-v1.0
```


## vLLM 서버 설정 

### vLLM 실행 예시
```bash
    python3 -m vllm.entrypoints.openai.api_server \
    --model TinyLlama/TinyLlama-1.1B-Chat-v1.0 \
    --gpu-memory-utilization 0.5 \
    --dtype float16 \
    --port 8000

```
### NVIDIA Chat 사용시 NVIDIA에서 지원하는 vLLM server 확인 후 실행 
```bash
    vllm serve microsoft/phi-3.5-vision-instruct  \
        --trust-remote-code \
        --max_model_len 16384 \
        --gpu-memory-utilization 0.8 \
        --enforce-eager \
        --port 9000
```

### 각 파라미터 설명 

- `microsoft/phi-3.5-vision-instruct`: HuggingFace에 있는 nulti-model 모델 이름 정의 
- `--trust-remote-code`: 모델의 커스텀 코드를 신뢰하겠다는 의미, Vision 모델처럼 코드가 포함된 모델에서는 필수
- `--max-model-len`: 최대 context length를 지정 
- `--gpu-memory-utillization`: GPU 메모리 사용 비율을 설정 
- `--enforce-eager`: 일부 모델에서 LazyTensor 대신 eager 모든 강제(디버깅, 호환성 목적)
- `--port 9000`: API server가 열리는 포트 지정 

### vLLM server의 LLM 호출 

In [1]:
from openai import OpenAI
import os 

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

client = OpenAI(
    base_url="http://localhost:8000/v1",  # vLLM 서버 주소
    api_key="EMPTY"  # 키 없어도 되지만 반드시 문자열 필요함
)

response = client.chat.completions.create(
    model = model_name,  # 서버에서 로딩한 모델 이름
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain the concept of diffusion models."}
    ],
    temperature=0.7,
)

print(response.choices[0].message.content)


Diffusion models are statistical algorithms that try to infer the underlying properties of a network based on the indirect influences of nodes on each other. The concept of diffusion models is based on the concept of nearest neighbors, which means that the influence of one node on another node can be strengthened or weakened by the influence of other nodes that are closer to both nodes.

By assuming that nodes have mutual influence on each other, the diffusion model can be used to infer the properties of the network. These properties can include the degree distribution, clustering coefficient, community structure, and centrality measures.

The most common diffusion models used in network analysis include the following:

1. Greedy Algorithm: This model assumes that the network is initially well-connected. The algorithm iterates through the network, adding new nodes and edges, until the network is fully connected.

2. Random Walk with Restart (RWAR) Algorithm: This model assumes that the

In [ ]:
client.completions.create(
    model = model_name,
    prompt = "Hello! How's it going?"
)

Completion(id='cmpl-c475f686f60a457ea0428a745f22264f', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=" I hope you are well. I think I'm missing you. You have", stop_reason=None, prompt_logprobs=None)], created=1754026683, model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=9, total_tokens=25, completion_tokens_details=None, prompt_tokens_details=None), service_tier=None, kv_transfer_params=None)

### vLLM과 LangChain 연동 

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(
    base_url = "http://localhost:8000/v1",
    api_key = 'EMPTY',
    model_name = model_name,
    temperature = 0.7
)

print(llm.predict("What's the capital of France?"))

C:\Users\user\AppData\Local\Temp\ipykernel_11636\4140960587.py:11: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm.predict("What's the capital of France?"))


The capital of France is Paris.


### vLLM과 LangChain 기반 간단한 Chatbot

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatOpenAI(
    base_url = "http://localhost:8000/v1",
    api_key = 'EMPTY',
    model_name = model_name,
    temperature = 0.7
)

messages = [
    SystemMessage(content = "You are a helpful assistant."),
    HumanMessage(content = "Hello, who won the World Cup in 2022?")
]

response = llm(messages)
print(f"Bot: {response.content}")

C:\Users\user\AppData\Local\Temp\ipykernel_11636\2703758605.py:16: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


Bot: The 2022 FIFA World Cup, the most prestigious annual international football tournament, was won by Argentina.


### Full-context

- 대화 메시지를 계속 쌓아서 전체를 프롬프트로 전달 
- context 길이가 너무 길어지면 느려지고, 비용이 증가 

### Running State 

- 대화 중 상태를 유지하는 것으로, 상태 변수를 별도로 만들어서 관리한다. 
- 대화 이력 중 중요한 부분만 부분적으로 유지하고 요약해서 관리 가능하다.
- `langgraph`로 memory 처리 

### Sliding Window

- 최근 N개의 메시지만 프롬프트로 전달
- 토큰 길이 제한에 효과적



In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

llm = ChatOpenAI(
    base_url = "http://localhost:8000/v1",
    api_key = 'EMPTY',
    model_name = model_name,
    temperature = 0.7
)

# 모든 대화 내용을 누적
# Full-context
history = [
    SystemMessage(content = "You are a helpful assistant"),
]

while True:
    user_input = input("User: ")
    history.append(HumanMessage(content = user_input))
    
    response = llm(history)
    print(f"Boat: {response.content}")

Boat: I don't have access to real-time weather updates, but according to current weather conditions, the weather today in your area is usually sunny and mild. The temperature may vary depending on elevation, but in general, the temperature is around 60°F (16°C) to 70°F (21°C). The wind direction is usually northerly or northeasterly, with gusts of up to 15 mph (24 km/h).


## NIM(NVIDIA Inference Microservice)

NIM은 엔터프라이즈 환경에서 AI 모델을 쉽고 안정적으로 배포하기 위한 엔비디아의 통합 솔루션으로, AI 모델을 컨테이너화된 마이크로서비스 형태로 제공하여 배포를 단순화하고, 최적화된 성능을 보장한다. Local에서 사용할 수 있으며 Local device에서의 최적화가 자동으로 이루어져있고, Privacy한 LLM을 구축할 수 있다. 

### NIM 특징 
- 컨테이너: AI모델과 필요한 모든 구성 요소를 컨테이너 형태로 패키징하여 클라우드, 데이터 센터, 그리고 로컬 등 어떤 환경에서도 일관되게 모델을 활용할 수 있다.
- 성능 최적화: TensorRT나 vLLM 등 최적화 엔진이 내장되어있어 모델의 추론 성능을 극대화 시킬 수 있다.
- 표준 API: 개발자들이 간단하게 모델을 애플리케이션에 설계 통합할 수 있다. 

### NVIDIA Chat

NIM의 기능을 랭체인과 연동하여 사용한다. 로컬에서 사용하더라도 클라우드에서 연산을 처리하므로, Large Model 들도 사용 가능하다. 

nvidia에서 지원하는 모델 중 `microsoft/phi-3.5-vision-instruct`ㅡ `google-gemma3` 등의 모델을 Serving가능하다. 

In [19]:
import os
from dotenv import load_dotenv

load_dotenv("../.././.env")

nvidia_api_key = os.getenv('NVIDIA_API_KEY')
print(f"NVIDIA API Key Loaded: {nvidia_api_key is not None}")

NVIDIA API Key Loaded: True


In [25]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content = "당신은 유용한 인공지능 비서입니다. 친절하게 답변해주세요"),
    HumanMessage(content = "안녕 gemma는 어떤 모델이야?")
]

llm = ChatNVIDIA(model = 'google/gemma-7b')

response = llm.invoke(messages)

print(response.content)


저는 자연어 처리 모델이며, Transformer 기반의 모델입니다. 이 기술은 기울임 모델을 사용하여 단어 간의 관계를 학습하고 문장의 의미를 이해합니다.


## Ollama

로컬 환경에서 LLM을 쉽게 실행할 수 있도록 설계된 사용자 친화적 도구이다. 복잡한 설정 없이도 몇 가지 명렬어만으로 로컬 pc에서 다양한 오픈소스 모델을 구동할 수 있다. 

- `ollama list` 명령어로 pull 되어 있는 LLM 확인 가능 
- `ollama run llama3`, `ollama pull gemma3-lateset` 로 실행 가능
- 클라우드나 외부 서버에 의존하지 않고, 개인 PC나 노트북에서 LLM을 구동하므로 데이터 프라이버시가 중요한 경우에 활용 가능하다. 

In [1]:
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(
    model = 'gemma3',
    base_url = 'http://localhost:11434',
    temperature = 0.7
)

response_ollama = llm_ollama.invoke("안녕하세요? 오늘 날씨는 어떤가요?")

print(response_ollama)

content='안녕하세요! 오늘 날씨는 지역에 따라 다릅니다. 어디에 계신가요? \n\n*   **서울:** 맑고 건조하며, 낮 최고기온은 28도까지 올라갑니다.\n*   **부산:** 구름이 조금 있으며, 낮 최고기온은 26도입니다.\n*   **대구:** 맑고 건조하며, 낮 최고기온은 30도입니다.\n\n좀 더 자세한 날씨 정보가 필요하시면, 계신 지역을 알려주세요. 😊' additional_kwargs={} response_metadata={'model': 'gemma3', 'created_at': '2025-08-03T06:30:14.0586027Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1950441700, 'load_duration': 82384300, 'prompt_eval_count': 19, 'prompt_eval_duration': 281028500, 'eval_count': 118, 'eval_duration': 1585276300, 'model_name': 'gemma3'} id='run--bbf3a08d-39d8-4f21-966c-59a614926559-0' usage_metadata={'input_tokens': 19, 'output_tokens': 118, 'total_tokens': 137}


In [4]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

model = ChatOllama(
    model = 'gemma3',
    base_url = 'http://localhost:11434',
    temperature = 0.7
)

# from_template는 미리 정의된 문자열 템플릿으로부터 프롬프트 객체 생성 
template = PromptTemplate.from_template(
    '''아래에 작성한 컨텍스트(context)를 기반으로 질문(question)에 대답하세요. 제공된 정보로 대답할 수 없는 질문이라면 '모르겠어요'라고 답하세요
    
Context : {context}

Question: {question}

Answer: '''
)

prompt = template.invoke({
    "context":"거대 언어 모델은 자연어처리 분양의 최신 발전을 이끌고 있다. 더 작은 모델보다 우수한 성능을 보이며 NLP 기능을 갖춘 애플리케이션을 개발하는 개발자들에게 매우 중요한 도구가 되었습니다.",
    'question':"거대 언어 모델은 무엇인가요?"
})

completion = model.invoke(prompt)
completion.content

'거대 언어 모델은 자연어 처리 분야의 최신 발전을 이끌고 있는 모델입니다. 더 작은 모델보다 우수한 성능을 보이며 NLP 기능을 갖춘 애플리케이션을 개발하는 개발자들에게 중요한 도구입니다.'